In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
light_df = pd.read_csv("data/대구 보안등 정보.csv",encoding='euc-kr')
child_df = pd.read_csv("data/대구 어린이 보호 구역 정보.csv",encoding='euc-kr')
cctv_df = pd.read_csv("data/대구 CCTV 정보.csv",encoding='euc-kr')
park_df = pd.read_csv("data/대구 주차장 정보.csv",encoding='euc-kr')


C:\Users\user\AppData\Local\Temp\ipykernel_11060\1564858603.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  light_df = pd.read_csv("data/대구 보안등 정보.csv",encoding='euc-kr')


In [109]:
park_df = park_df.drop('주차장관리번호',axis=1 )
park_df = park_df.drop('데이터기준일자',axis=1 )
park_df = park_df.drop('특기사항',axis=1 )
park_df = park_df.drop('부제시행구분',axis=1 )
park_df = park_df.drop('소재지도로명주소',axis=1 )
park_df = park_df.drop(columns=['주차기본시간','주차기본요금','추가단위시간','추가단위요금','1일주차권요금적용시간','1일주차권요금','월정기권요금','결제방법'],axis=1 )

park_df['소재지지번주소'] = park_df['소재지지번주소'].str.replace('대구광역시','')
# 1번 대구광역시 자르기
park_df = park_df.dropna(subset=['소재지지번주소'])
park_df['주차장명'] = 1

park_df['소재지지번주소'] = park_df['소재지지번주소'].apply(lambda x: ' '.join(x.split()[:2]) if len(x.split()) > 1 else x)
park_df['소재지지번주소'] = park_df['소재지지번주소'].apply(lambda x: x.split()[-1])
#공백기준 2번째 까지 남기고 자르기
park_df = park_df.dropna()

# 3번 동읍면 제외하고 뒤에 다 자름
park_df['소재지지번주소'] = park_df['소재지지번주소'].replace(to_replace=r'(\d+)', value='', regex=True)
park_df['소재지지번주소'] = park_df['소재지지번주소'].str.replace('·', '')
park_df['소재지지번주소'] = park_df['소재지지번주소'].str.replace('.', '')
park_df['소재지지번주소'] = park_df['소재지지번주소'].str.replace('-', '').str.rstrip('가')

park_df = park_df.groupby('소재지지번주소').agg({
    '주차장명': 'size',
     # 주차장명의 중복되는 행들의 count
    '주차장구분': lambda x: x.mode().iloc[0],
    '주차장유형': lambda x: x.mode().iloc[0],
    '주차구획수': 'mean',
    '급지구분': 'mean',
    '운영요일': lambda x: x.mode().iloc[0],
    '평일운영시작시각': lambda x: x.mode().iloc[0],
    '평일운영종료시각': lambda x: x.mode().iloc[0],
    '토요일운영시작시각': lambda x: x.mode().iloc[0],
    '토요일운영종료시각': lambda x: x.mode().iloc[0],
    '공휴일운영시작시각': lambda x: x.mode().iloc[0],
    '공휴일운영종료시각': lambda x: x.mode().iloc[0],
    '요금정보': lambda x: x.mode().iloc[0],
    '위도': 'mean',
    '경도': 'mean'
}).reset_index()



list = ['무인교통단속카메라관리번호','시도명','시군구명','도로노선명' ,'소재지도로명주소','위도','경도','단속구간위치구분','과속단속구간길이','설치연도' ,'도로노선번호' ,'보호구역구분']
cctv_df = cctv_df.drop(columns=list, axis=1)

cctv_df['제한속도'] = cctv_df['제한속도'].astype('object')
cctv_df['단속구분'] = cctv_df['단속구분'].astype('object')
# cctv_df['보호구역구분'] = cctv_df['보호구역구분'].astype('object')
cctv_df['도로노선방향'] = cctv_df['도로노선방향'].astype('object')
cctv_df['설치장소'] = 1

#1번 대구광역시 자르기
cctv_df['소재지지번주소'] = cctv_df['소재지지번주소'].str.replace('대구광역시','')
cctv_df = cctv_df.dropna(subset=['소재지지번주소'])

#2번 공백기준 두번째까지 남기고 자르기
cctv_df['소재지지번주소'] = cctv_df['소재지지번주소'].apply(lambda x: ' '.join(x.split()[:2]) if len(x.split()) > 1 else x)
cctv_df['소재지지번주소'] = cctv_df['소재지지번주소'].apply(lambda x: x.split()[-1])
# 동일하게
cctv_df['소재지지번주소'] = cctv_df['소재지지번주소'].str.replace(r'\d{2,}', '')


cctv_df['소재지지번주소'] = cctv_df['소재지지번주소'].replace(to_replace=r'(\d+)', value='', regex=True)
cctv_df['소재지지번주소'] = cctv_df['소재지지번주소'].str.replace('·', '')
cctv_df['소재지지번주소'] = cctv_df['소재지지번주소'].str.replace('.', '')
cctv_df['소재지지번주소'] = cctv_df['소재지지번주소'].str.replace('-', '').str.rstrip('가')

cctv_df = cctv_df.groupby('소재지지번주소').agg({
    '설치장소': 'size',
    '도로종류': lambda x: x.mode().iloc[0],
    '도로노선방향': lambda x: x.mode().iloc[0],
    # '보호구역구분': lambda x: x.mode().iloc[0],
    '제한속도': lambda x: x.mode().iloc[0],
    '단속구분': lambda x: x.mode().iloc[0]
}).reset_index()






list = ['데이터기준일자','소재지도로명주소','관리기관명','CCTV설치대수' ,'보호구역도로폭','위도','경도']
child_df = child_df.drop(columns=list, axis=1)

#1번 대구광역시 자르기
child_df['소재지지번주소'] = child_df['소재지지번주소'].str.replace('대구광역시','')
child_df = child_df.dropna(subset=['소재지지번주소'])

#2번 공백기준 두번째까지 남기고 자르기
child_df['소재지지번주소'] = child_df['소재지지번주소'].apply(lambda x: ' '.join(x.split()[:2]) if len(x.split()) > 1 else x)
child_df['소재지지번주소'] = child_df['소재지지번주소'].apply(lambda x: x.split()[-1])
# 동일하게
child_df['소재지지번주소'] = child_df['소재지지번주소'].str.replace(r'\d{2,}', '')


child_df['소재지지번주소'] = child_df['소재지지번주소'].replace(to_replace=r'(\d+)', value='', regex=True)
child_df['소재지지번주소'] = child_df['소재지지번주소'].str.replace('·', '')
child_df['소재지지번주소'] = child_df['소재지지번주소'].str.replace('.', '')
child_df['소재지지번주소'] = child_df['소재지지번주소'].str.replace('-', '').str.rstrip('가')
child_df['대상시설명'] = 1


child_df = child_df.groupby('소재지지번주소').agg({
    '대상시설명': 'size',
     # 주차장명의 중복되는 행들의 count
    'CCTV설치여부': lambda x: x.mode().iloc[0],
    '관할경찰서명': lambda x: x.mode().iloc[0],
    '시설종류': lambda x: x.mode().iloc[0]


}).reset_index()








list = ['보안등위치명','소재지도로명주소','위도','경도','설치연도','설치형태']
light_df = light_df.drop(columns=list, axis=1)

#1번 대구광역시 자르기
light_df['소재지지번주소'] = light_df['소재지지번주소'].str.replace('대구광역시','')
light_df = light_df.dropna(subset=['소재지지번주소'])

#2번 공백기준 두번째까지 남기고 자르기
light_df['소재지지번주소'] = light_df['소재지지번주소'].apply(lambda x: ' '.join(x.split()[:2]) if len(x.split()) > 1 else x)
light_df['소재지지번주소'] = light_df['소재지지번주소'].apply(lambda x: x.split()[-1])
# 동일하게

# light_df['소재지지번주소'] = light_df['소재지지번주소'].apply(lambda x: re.sub(r'\d{2,}-\d+', '', x))
# light_df['소재지지번주소'] = light_df['소재지지번주소'].apply(lambda x: re.sub(r'\d{2,}', '', x))

# 3번 동읍면 제외하고 뒤에 다 자름
light_df['소재지지번주소'] = light_df['소재지지번주소'].replace(to_replace=r'(\d+)', value='', regex=True)
light_df['소재지지번주소'] = light_df['소재지지번주소'].str.replace('·', '')
light_df['소재지지번주소'] = light_df['소재지지번주소'].str.replace('.', '')
light_df['소재지지번주소'] = light_df['소재지지번주소'].str.replace('-', '').str.rstrip('가')


light_df = light_df.groupby('소재지지번주소').agg({
    '설치개수': 'size'
}).reset_index()

# light_df['소재지지번주소'] = light_df['소재지지번주소'].str.replace(r'\d+|-', '')
#여기부터 다시 생각
# light_df['소재지지번주소'] = light_df['소재지지번주소'].replace(to_replace=r'(\d+)', value='', regex=True)
#숫자만 나오는 부분 자르기



In [ ]:
cctv_df.to_csv("data/cctv_df.csv")
child_df.to_csv("data/child_df.csv")
light_df.to_csv("data/light_df.csv")
park_df.to_csv("data/park_df.csv")


In [114]:
light_df.info()
for c in light_df.columns:
    un = light_df[c].unique()
    print(f"칼럼 {c}의 유니크 값 {un}")
    print()

for c in child_df.columns:
    un = child_df[c].unique()
    print(f"칼럼 {c}의 유니크 값 {un}")
    print()

for c in cctv_df.columns:
    un = cctv_df[c].unique()
    print(f"칼럼 {c}의 유니크 값 {un}")
    print()

for c in park_df.columns:
    un = park_df[c].unique()
    print(f"칼럼 {c}의 유니크 값 {un}")
    print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   소재지지번주소  175 non-null    object
 1   설치개수     175 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.9+ KB
칼럼 소재지지번주소의 유니크 값 ['' '가창면' '각산동' '갈산동' '감삼동' '검단동' '검사동' '계산동' '고산동' '고성동' '공평동' '관음동'
 '괴전동' '교동' '구암동' '구지면' '국우동' '금강동' '금호동' '남산동' '남성로' '남일동' '내곡동' '내당동'
 '내동' '노곡동' '노원동' '논공읍' '능성동' '다사읍' '달성동' '대곡동' '대림동' '대명동' '대봉동' '대신동'
 '대안동' '대천동' '대현동' '덕곡동' '덕산동' '도남동' '도동' '도원동' '도의리' '도평로' '도학동' '동구'
 '동내동' '동문동' '동변동' '동산동' '동성로' '동인동' '동일동' '동천동' '동호동' '두류동' '두산동' '둔산동'
 '만촌동' '매여동' '매천동' '문화동' '미곡동' '미대동' '방촌동' '백안동' '범물동' '범어동' '복현동' '본동'
 '본리동' '봉덕동' '봉무동' '봉산동' '부동' '부동길' '북내동' '북성로' '불로동' '비산동' '사복동' '사일동'
 '산격동' '삼덕동' '상덕동' '상동' '상매동' '상서동' '상인동' '상중이동' '서내동' '서문로' '서변동' '서성로'
 '서야동' '서호동' '성당동' '송정동' '송촌리' '송현동' '수동' '수성가동' '수창동' '숙천동' '시장북로' '신기동'
 '신당동' '신무동' '신서동' '신암동' '신용동' 